In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [12]:
# Image dimensions
IMG_SIZE = (48, 48)
BATCH_SIZE = 32

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=20, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load train and test datasets
train_generator = train_datagen.flow_from_directory(
    "dataset/fer/train", 
    target_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    color_mode="rgb", 
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    "dataset/fer/test", 
    target_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    color_mode="rgb", 
    class_mode="categorical"
)

num_classes = len(train_generator.class_indices)
print("Number of classes:", num_classes)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Number of classes: 7


In [13]:
# Load VGG16 (pretrained on ImageNet)
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(48, 48, 3))

# Freeze base model layers
base_model.trainable = False

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(256, activation="relu"),
    Dropout(0.5),
    Dense(num_classes, activation="softmax")  # Output layer
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss="categorical_crossentropy", 
              metrics=["accuracy"])

# Model summary
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 1, 1, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,847,815 (56.64 MB)

 Trainable params: 133,127 (520.03 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [14]:
EPOCHS = 15

history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS
)

Epoch 1/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 151s 168ms/step - accuracy: 0.2364 - loss: 1.8979 - val_accuracy: 0.3217 - val_loss: 1.6913
Epoch 2/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 149s 166ms/step - accuracy: 0.2901 - loss: 1.7434 - val_accuracy: 0.3373 - val_loss: 1.6871
Epoch 3/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 149s 166ms/step - accuracy: 0.2956 - loss: 1.7349 - val_accuracy: 0.3447 - val_loss: 1.6561
Epoch 4/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 150s 167ms/step - accuracy: 0.3009 - loss: 1.7217 - val_accuracy: 0.3434 - val_loss: 1.6513
Epoch 5/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 149s 166ms/step - accuracy: 0.3046 - loss: 1.7233 - val_accuracy: 0.3422 - val_loss: 1.6552
Epoch 6/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 150s 167ms/step - accuracy: 0.3046 - loss: 1.7147 - val_accuracy: 0.3390 - val_loss: 1.6476
Epoch 7/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 153s 170ms/step - accuracy: 0.3139 - loss: 1.7149 - val_accuracy: 0.3431 - val_loss: 1.6551
Epoch 8/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 151s 169ms/step - accuracy: 0.3066 -

In [15]:
# Get train and test accuracies
train_loss, train_acc = model.evaluate(train_generator)
test_loss, test_acc = model.evaluate(test_generator)

print(f"Train Accuracy: {train_acc * 100:.2f}%")
print(f"Test Accuracy: {test_acc * 100:.2f}%")


898/898 ━━━━━━━━━━━━━━━━━━━━ 120s 134ms/step - accuracy: 0.3299 - loss: 1.6573
225/225 ━━━━━━━━━━━━━━━━━━━━ 29s 130ms/step - accuracy: 0.3432 - loss: 1.6261
Train Accuracy: 32.63%
Test Accuracy: 33.63%


In [17]:
# Save model
model.save("trained_models/vgg16_fer_15_epochs.keras")

# # Load model later
# from tensorflow.keras.models import load_model
# model = load_model("vgg16_fer.h5")


In [18]:
from sklearn.metrics import classification_report

# Assuming you have the true labels and predicted labels for both train and test datasets
train_predictions = model.predict(train_generator)
test_predictions = model.predict(test_generator)

# Convert predictions to class labels (if necessary, depends on your model output)
train_predictions_labels = np.argmax(train_predictions, axis=1)
test_predictions_labels = np.argmax(test_predictions, axis=1)

# Assuming your labels are categorical and represented as integers
print("Train Classification Report:")
print(classification_report(train_generator.classes, train_predictions_labels))

print("Test Classification Report:")
print(classification_report(test_generator.classes, test_predictions_labels))

# You can also extract individual metrics:
# For train
train_report = classification_report(train_generator.classes, train_predictions_labels, output_dict=True)
print(f"Train Precision: {train_report['weighted avg']['precision']:.2f}")
print(f"Train Recall: {train_report['weighted avg']['recall']:.2f}")
print(f"Train F1-Score: {train_report['weighted avg']['f1-score']:.2f}")

# For test
test_report = classification_report(test_generator.classes, test_predictions_labels, output_dict=True)
print(f"Test Precision: {test_report['weighted avg']['precision']:.2f}")
print(f"Test Recall: {test_report['weighted avg']['recall']:.2f}")
print(f"Test F1-Score: {test_report['weighted avg']['f1-score']:.2f}")


898/898 ━━━━━━━━━━━━━━━━━━━━ 126s 140ms/step
225/225 ━━━━━━━━━━━━━━━━━━━━ 30s 133ms/step
Train Classification Report:
              precision    recall  f1-score   support

           0       0.14      0.00      0.01      3995
           1       0.00      0.00      0.00       436
           2       0.15      0.02      0.03      4097
           3       0.25      0.76      0.38      7215
           4       0.18      0.06      0.09      4965
           5       0.16      0.07      0.10      4830
           6       0.11      0.09      0.10      3171

    accuracy                           0.23     28709
   macro avg       0.14      0.14      0.10     28709
weighted avg       0.17      0.23      0.14     28709

Test Classification Report:
              precision    recall  f1-score   support

           0       0.20      0.01      0.02       958
           1       0.00      0.00      0.00       111
           2       0.14      0.02      0.04      1024
           3       0.24      0.73      0

/Users/ayushidubey/Desktop/College/Semester-6/NNDL/labs/project/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ayushidubey/Desktop/College/Semester-6/NNDL/labs/project/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ayushidubey/Desktop/College/Semester-6/NNDL/labs/project/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use 